The following packages need to be downloaded:

Astroquery,
numpy >= 1.14,
astropy (>=3.1),
requests,
keyring,
Beautiful Soup,
html5lib,
six,
curl,
pytest-astropy,
pyregion,



Run this section first to load all the prerequisite packages.

In [ ]:
from astroquery.vizier import Vizier
import requests
from bs4 import BeautifulSoup
from astroquery.simbad import Simbad
import numpy as np
from astropy.table import Table
from astroquery.xmatch import XMatch
from astropy.io import ascii
from astropy import units as u
import ipython_bell

Please run this section of the code to download the data on constellation borders. The csv file will contain the ra and dec that must be plotted (scatter) to obtain the borders of constellations

In [ ]:
#First we import astroquery and remove the limit on rows.

from astroquery.vizier import Vizier
catalogue_list = Vizier.find_catalogs("Constellation")#This loads all the catalogs by the keyword "constellation"
Vizier.ROW_LIMIT = -1

catalog = Vizier.get_catalogs("VI/49")#This selects the 2nd of three tables that were found by the key-word search. This table contains relevant data
catalog[1].write("constellation_borders.csv",format="csv",overwrite="True")#creates a csv file
%bell

The output file generated by the above script will be stored in your working directory. to load tables directly from the Internet using URLs. For example, download tables from Vizier catalogues in CDS format ('ascii.cds'):

Run this section to download the ra,dec of messier objects

In [ ]:
#PARSING THROUGH HYPERLEDA
#will add all mags into an array
mags = []

import requests
from bs4 import BeautifulSoup
for i in range(110):
    page = requests.get("http://leda.univ-lyon1.fr/ledacat.cgi?o=M{}".format(i+1))
    soup = BeautifulSoup(page.content,'html.parser')
    if len(soup.find_all('table',attrs={"class":"datatable"}))!=0:
        table_rel = soup.find_all('table',attrs={"class":"datatable"})[1]
        index = 0
        for rows in table_rel.find_all('tr'):
            if rows.find_all('a', attrs = {"title":"Total V-magnitude"}) != []:
                            mag_value = rows.find_all('td')[1].get_text().split()[0]
                            mags.append(mag_value)
                            index = index + 1
                            break
            if rows == table_rel.find_all('tr')[-1] and index == 0:
                mags.append('-')
                break
    else:
        mags.append('-')
        continue

#NEED TO ADD THESE VALUES TO THE MESSIER TABLE

In [ ]:

#Import Simbad query
from astroquery.simbad import Simbad

#the next three lines are to simplify the output table
Simbad.reset_votable_fields()
Simbad.remove_votable_fields('coordinates')
Simbad.add_votable_fields('otype(S)', 'ra(d;A;ICRS;J2000;2000)', 'dec(d;D;ICRS;J2000;2000)','distance','flux(B)','flux(V)','flux_unit(V)')#These are all the columns added to the table

result_table = Simbad.query_catalog("Messier")#This asks the SIMBAD database to list all objects from the messier catalog
result_table['V Mag (from HyperLeda/SED)'] = mags#adding v mags
result_table.write("messier_objects.csv",format="csv",overwrite="True")#creates a csv file

Simbad.reset_votable_fields()#renders the prev changes to simbad class temporary.

NGC Catalogue

In [ ]:

from astroquery.vizier import Vizier
v = Vizier(columns = ['Name','Type','mag','Const','RA (deg)','Dec (deg)'])#Columns added to table
v.ROW_LIMIT = -1
result_table = v.get_catalogs("VII/118/ngc2000")[0]
result_table.write("NGC.csv",format="csv",overwrite="True")

Hipparcos Catalogue

In [ ]:
#Part of Hipparcos catalogue below 6 mags
#from astroquery.vizier import Vizier
#v = Vizier(columns = ['HD','_RAJ2000','DECJ2000','B-V','Vmag','SpType','Plx'], column_filters = {"Vmag":"<=6"})
#v.ROW_LIMIT = -1
#v.TIMEOUT = 1000
#catalog_stars = v.get_catalogs("I/239/hip_main")[0]
#catalog_stars.write("Hipparcos_1.csv",format="csv",overwrite = "True")


In [ ]:
#Part of Hipparcos catalogue between 6 to 8 mags
#from astroquery.vizier import Vizier
#v = Vizier(columns = ['HD','_RAJ2000','DECJ2000','B-V','Vmag','SpType','Plx'], column_filters = {"Vmag":"6..8"})
#v.ROW_LIMIT = -1
#v.TIMEOUT = 1000
#catalog_stars = v.get_catalogs("I/239/hip_main")[0]
#catalog_stars.write("Hipparcos_2.csv",format="csv",overwrite = "True")

Tycho-2 Catalogue

In [19]:
v = Vizier(columns = ['HIP','TYC','HD','_RAJ2000','DECJ2000','B-V','Vmag','Plx'], column_filters = {"Vmag":"<6"})
v.ROW_LIMIT = -1
v.TIMEOUT = 1000
catalog_stars = v.get_catalogs("I/239/tyc_main")[0]
catalog_stars.write("tycho_1.csv",format="csv",overwrite = "True")
%bell

In [20]:
v = Vizier(columns = ['HIP','TYC','HD','_RAJ2000','DECJ2000','B-V','Vmag','Plx'], column_filters = {"Vmag":"6..9"})
v.ROW_LIMIT = -1
v.TIMEOUT = 1000
catalog_stars = v.get_catalogs("I/239/tyc_main")[0]
catalog_stars.write("tycho_2.csv",format="csv",overwrite = "True")
%bell

In [23]:
v = Vizier(columns = ['HIP','TYC','HD','_RAJ2000','DECJ2000','B-V','Vmag','Plx'], column_filters = {"Vmag":"9..10"})
v.ROW_LIMIT = -1
v.TIMEOUT = 1000
catalog_stars = v.get_catalogs("I/239/tyc_main")[0]
catalog_stars.write("tycho_3.csv",format="csv",overwrite = "True")
%bell

In [25]:
v = Vizier(columns = ['HIP','TYC','HD','_RAJ2000','DECJ2000','B-V','Vmag','Plx'], column_filters = {"Vmag":"10..12"})
v.ROW_LIMIT = -1
v.TIMEOUT = 1000
catalog_stars = v.get_catalogs("I/239/tyc_main")[0]
catalog_stars.write("tycho_4.csv",format="csv",overwrite = "True")
%bell

Gaia DR2

In [55]:
v = Vizier(columns = ['HIP','TYC','_RAJ2000','DECJ2000','<Gmag>','VTmag','BTmag','Plx'], column_filters = {"Vmag":"12..13"})
#(-0.01760-0.006860*(GBP-GRP)-0.1732*(GBP-GRP)2)
v.ROW_LIMIT = 50
v.TIMEOUT = 1000
catalog_stars = v.get_catalogs("I/337/tgasptyc")[0]
vmag = catalog_stars['VTmag']-0.09*(catalog_stars['BTmag']-catalog_stars['VTmag'])
catalog_stars['Vmag'] = vmag
plx = catalog_stars['Plx']
catalog_stars.remove_columns(['VTmag','BTmag','__Gmag_','Plx'])
catalog_stars['Plx'] = plx
catalog_stars.write("gaia_tgas_1.csv",format="csv",overwrite = "True")
#%bell

If you want to cross match the hipparcos and nomad files, you can use the XMatch module. DONT RUN THESE FOR THE TIME BEING

In [ ]:
from astroquery.xmatch import XMatch
from astropy import units as u
#print(XMatch.is_table_available("I/239/hip_main"))

table = XMatch.query(cat1=open('NOMAD_1.csv'), cat2="vizier:I/239/hip_main", max_distance=18 * u.arcsec, colRA1='RAJ2000', colDec1='DEJ2000')
table.write("NOMAD_1_join.csv",format='csv', overwrite = 'True')

In [ ]:
from astropy.table import Table
table = Table.read('NOMAD_1_join.csv')
table2 = Table.read('NOMAD_1.csv')
ra1 = table['RAJ2000']
ra2 = table2['RAJ2000']
count = 0
for elt in ra2:
    print(elt in ra1)

Bright Star Catalogue

In [ ]:
from astroquery.vizier import Vizier
catalog_strs = Vizier.get_catalogs("V/50")
Vizier.ROW_LIMIT = -1
catalog_strs[0].write("BSC5P.csv",format="csv",overwrite="True")

Star names.

Step 1: Extract the names of the stars from the IAU website
        

In [ ]:
names = []
hr_no = []
const = []
ident = []
vmag = []
ra = []
dec = []


page = requests.get("https://www.iau.org/public/themes/naming_stars/")
soup_page = BeautifulSoup(page.content,'html.parser')
table = soup_page.find_all("table",attrs = {"class":"table sortable"})[0]
table_rows = table.find_all("tr")[1:]
for rows in table_rows:
    columns = rows.find_all("td")
    names.append(columns[0].get_text())
    hr_no.append(columns[1].get_text())
    ident.append(columns[2].get_text())
    const.append(columns[3].get_text())
    vmag.append(columns[6].get_text())
    ra.append(columns[7].get_text())
    dec.append(columns[8].get_text())
    

ascii.write([names,hr_no,ident,const,vmag,ra,dec],"IAU_names.csv",format="csv",names = ['Name','HR','ID','Const','Vmag','RA','Dec'],overwrite="True")


In [ ]:
file = Table.read("IAU_names_join.csv",format="csv")
name_new = np.array(file['Name'])
for items in names:
    print(np.where(items in name_new))


In [ ]:
v = Vizier(columns = ["HIP","TYC2","RAJ2000","DEJ2000","BTmag","VTmag"],column_filters={"VTmag":"<=6"})
v.ROW_LIMIT=-1
v.TIMEOUT = 1000
catalog_strs = v.get_catalogs("I/337/tgasptyc")[0]
catalog_strs.write("gaia.csv", format = "csv", overwrite = "True")